# Exercises

## Overview

### Questions

* What structures to systems of hard spherocylinders form?
* What aspect ratio leads to a nematic phase?
* How can I quantify the nematic phase with an order parameter?

### Objectives

* Show how to write a hard spherocylinder simulation in reusable functions.
* Demonstrate nematic ordering and compute the nematic order parameter for the system.
* Suggest exercises for further exploration.

## Boilerplate code

In [ ]:
import hoomd
import math
import itertools
import numpy
import copy
import gsd.hoomd
import freud
import matplotlib
import IPython
import rowan
%matplotlib inline
matplotlib.style.use('ggplot')

The `render` function in the next (hidden) cell will render a snapshot using **fresnel**.

<div class="alert alert-info">
    This is not intended as a full tutorial on <b>fresnel</b> - see the <a href="https://fresnel.readthedocs.io/">fresnel user documentation</a> if you would like to learn more.
</div>

In [ ]:
import fresnel

device = fresnel.Device()
tracer = fresnel.tracer.Path(device=device, w=250, h=250)

def render(snapshot, params):
    box_L = snapshot.configuration.box[0]

    scene = fresnel.Scene(device)
    geometry = fresnel.geometry.Cylinder(scene,
                                         N=snapshot.particles.N,
                                         radius=params['D']/2)
    geometry.material = fresnel.material.Material(color=fresnel.color.linear([252/255, 209/255, 1/255]),
                                                  roughness=0.5)
    
    top = snapshot.particles.position[:] + rowan.rotate(snapshot.particles.orientation, [0,0,params['L']/2])
    bottom = snapshot.particles.position[:] + rowan.rotate(snapshot.particles.orientation, [0,0,-params['L']/2])
    
    geometry.points[:,0,:] = top
    geometry.points[:,1,:] = bottom
    geometry.outline_width = 0.05
    box = fresnel.geometry.Box(scene, snapshot.configuration.box, box_radius=.04)
    
    scene.lights = [fresnel.light.Light(direction=(0, 0, 1), color=(0.8, 0.8, 0.8), theta=math.pi),
                    fresnel.light.Light(direction=(1, 1, 1), color=(1.1, 1.1, 1.1), theta=math.pi / 3)]
    scene.camera = fresnel.camera.orthographic(position=(0, -box_L-1, 0),
                                               look_at=(0, 0, 0),
                                               up=(0, 0, 1),
                                               height=box_L+params['L'])
    scene.background_color = (1,1,1)
    return tracer.sample(scene, samples=100)

## Workflow stages

The hard spherocylinder simulation script follows the same layout as that in the hard sphere crystallization tutorial with **initialization**, **randomization**, **compression**, **equilibration**, and **analysis** steps.
First, let's define the parameters of a hard spherocyliner system:
* `D`: The diameter of the spherocylinder.
* `L`: The length of the cylinder (**not** including the hemi-spherical caps).
* `rho_star`: Reduced density in the range 0-1 (as defined by [Bolhuis and Frenkel](https://dx.doi.org/10.1063/1.473404)).
* `seed`: Random number seed for MC simulation.
* `N_particles`: Number of particles to place in the simulation box.

In [ ]:
example_params = dict(D=1, L=4, rho_star=0.3, seed=1, N_particles=100)

You will pass this dictionary into each function that needs it.

The first function makes the HPMC integrator based on the given parameters.
Use the **ConvexSpheropolyhedron** geometry class to define spherocylinders.
In this case, a 2-vertex "spheropolyhedron" with vertices at [0,0,+/-L/2] is a spherocylinder.

In [ ]:
def make_hard_spherocylinder_integrator(params):
    mc = hoomd.hpmc.integrate.ConvexSpheropolyhedron(seed=params['seed'])
    mc.shape['spherocylinder'] = dict(vertices=[[0,0,-params['L']/2],
                                                [0,0,params['L']/2]],
                                      sweep_radius=params['D']/2)
    return mc

Create the initial condition by placing `N_particles` spherocylinders on a square grid in a box of height `2*(L+D)` so that they do not overlap and have plenty of space to randomize.
Writes this configuration out to `lattice.gsd`:

In [ ]:
def initialize(params):
    snapshot = gsd.hoomd.Snapshot()
    snapshot.particles.N = params['N_particles']
    
    spacing = params['D'] * 1.1
    K = math.ceil(params['N_particles']**(1/2))
    L = K * spacing
    x = numpy.linspace(-L / 2, L / 2, K, endpoint=False)
    position_2d = list(itertools.product(x, repeat=2))
    position_2d = position_2d[0:params['N_particles']]
    
    snapshot.particles.position = numpy.zeros(shape=(params['N_particles'], 3))
    snapshot.particles.position[:,0:2] = position_2d
    snapshot.particles.orientation = [1,0,0,0]*params['N_particles']
    snapshot.particles.types = ['spherocylinder']
    snapshot.configuration.box = [L, L, 2*(params['L'] + params['D']), 0, 0, 0]
    
    with gsd.hoomd.open(name='lattice.gsd', mode='wb') as f:
        f.append(snapshot)

The next function reads in `lattice.gsd` and runs HPMC simulations to randomize the positions and orientations of the particles and writes `random.gsd`:

In [ ]:
def randomize(params):
    cpu = hoomd.device.CPU()
    sim = hoomd.Simulation(device=cpu)
    sim.operations.integrator = make_hard_spherocylinder_integrator(params)
    sim.create_state_from_gsd(filename='lattice.gsd')
    sim.run(10e3)
    hoomd.write.GSD.write(state=sim.state, filename='random.gsd')

The next function reads in `random.gsd` and compresses the system to the target density and writes `compressed.gsd`.

In [ ]:
def compress(params):
    cpu = hoomd.device.CPU()
    sim = hoomd.Simulation(device=cpu)
    sim.operations.integrator = make_hard_spherocylinder_integrator(params)
    sim.create_state_from_gsd(filename='random.gsd')

    rho_c = 2/(math.sqrt(2) + (params['L']/params['D'])*math.sqrt(3))
    rho = rho_c * params['rho_star']
    box_V = sim.state.N_particles / rho   
    box_L=box_V**(1/3)
    final_box = hoomd.Box.cube(box_L)
        
    compress = hoomd.hpmc.update.QuickCompress(trigger=hoomd.trigger.Periodic(10), seed=params['seed'],
                                               target_box = final_box)
    sim.operations.updaters.append(compress)
    
    tune = hoomd.hpmc.tune.MoveSize.scale_solver(moves=['a', 'd'],
                                                 target=0.2,
                                                 trigger=hoomd.trigger.Periodic(10),
                                                 max_translation_move=0.2,
                                                 max_rotation_move=0.2)
    sim.operations.tuners.append(tune)   
   
    while not compress.complete and sim.timestep < 5e4:
        sim.run(1000)
    
    if not compress.complete:
        raise RuntimeError("Compression failed to complete")
    
    hoomd.write.GSD.write(state=sim.state, filename='compressed.gsd')

The next function reads in `compressed.gsd` and equilibrates the system over many steps, writing out `trajectory.gsd`:

In [ ]:
def equilibrate(params):
    cpu = hoomd.device.CPU()
    sim = hoomd.Simulation(device=cpu)
    sim.operations.integrator = make_hard_spherocylinder_integrator(params)
    sim.create_state_from_gsd(filename='compressed.gsd')

    gsd = hoomd.write.GSD(filename='trajectory.gsd',
                      trigger=hoomd.trigger.Periodic(1000),
                      mode='wb')
    sim.operations.writers.append(gsd)
    
    tune = hoomd.hpmc.tune.MoveSize.scale_solver(moves=['a', 'd'],
                                                 target=0.2,
                                                 trigger=hoomd.trigger.And(
                                                     [hoomd.trigger.Periodic(100),
                                                      hoomd.trigger.Before(sim.timestep + 5000)]))
    sim.operations.tuners.append(tune)   

    logger = hoomd.logging.Logger(categories=['scalar', 'string'])
    logger.add(sim, quantities=['timestep', 'final_timestep', 'tps'])
    table = hoomd.write.Table(trigger=hoomd.trigger.Periodic(period=5000),
                              logger=logger)
    sim.operations.writers.append(table)

    sim.run(200e3)

The next function analyzes the trajectory and ...:

In [ ]:
def analyze(params):
    with gsd.hoomd.open('trajectory.gsd') as traj:
        nematic = freud.order.Nematic([0, 0, 1])
        nematic_order = []
        for frame in traj:
            nematic.compute(frame.particles.orientation)
            nematic_order.append(nematic.order)

        IPython.display.display(render(traj[-1], params))
            
    fig = matplotlib.figure.Figure(figsize=(10, 6.18))
    ax = fig.add_subplot()
    ax.plot(nematic_order)
    ax.set_xlabel('frame')
    ax.set_ylabel('average nematic order parameter')
    ax.set_ylim([0, 1])
    IPython.display.display(fig)

The final function combines the `initialize`, `randomize`, `compress`, and `equilibrate` steps into one function.

In [ ]:
def run_simulation(params):
    initialize(params)
    randomize(params)
    compress(params)
    equilibrate(params)

## Run the simulation

Call the `run_simulation` function defined above to run the whole simulation with the given parameters:

<div class="alert alert-warning">
This cell will take a few minutes to complete.
</div>

In [ ]:
sim_params = dict(D=1, L=5, rho_star=0.53, seed=1, N_particles=50)
run_simulation(sim_params)

Analyze the results of the simulation:

In [ ]:
analyze(sim_params)

## Exercises

Try changing the simulation parameters above and rerunning the simulation.

* How does the system behave at lower densities (rho_star)?
* At what density does nematic order start to appear?
* Plot the nematic order in the final frame vs `rho_star`. How would you classify the nematic transition?
* How does the system behave with smaller values for L/D?

## Additional Simulations

This notebook initializes the system in a random configuration and allows it to self-assemble. 
You can also use simulations to explore melting behavior and the (meta-)stability of particular structures at higher densities where self-assembly simulations tend to jam.
Modify the **initialization** step to place several aligned layers of spherocylinders in a smectic structure at the target density and remove the compression step (hint: use a box with Lx == Ly and Lz > Lx).
* At what packing fraction does the L/D=5 system melt into a nematic structure? A fluid?
* Which type of simulation takes more timesteps to equilibrate: self-assembly or melting?

[Previous section](07-Analyzing-Trajectories.ipynb).